In [56]:
import pandas as pd
import re
import unicodedata
import itertools

In [57]:
# ==============================================================================
# ETAPA 1: CRIAÇÃO DE DADOS FICTÍCIOS
# ==============================================================================

# Dados fictícios para simular as reclamações
data = {
    'Texto_BC': [
        "Fui até a agencia para falar com o gerente sobre o meu problema, mas nao tive retorno algum sobre o caso.",
        "Estou há semanas aguardando um contato da agencia. Ninguém me liga, o gerente nunca está disponível.",
        "Total falta de feedback da minha agencia. Preciso de uma solução e não consigo falar com ninguém.",
        "O aplicativo do banco é ótimo, mas o atendimento na agencia deixa a desejar. O gerente prometeu um retorno que nunca aconteceu.",
        "Recebi um email sobre um novo produto, mas quando liguei para a central, não souberam me informar. Não tem a ver com a agencia.",
        "Problema resolvido rapidamente pelo telefone, não precisei ir ao banco nem falar com o gerente.",
        "A agencia está sempre lotada. Pedi um retorno do meu gerente sobre o investimento e até agora nada.",
        "Não obtive nenhuma resposta da agencia sobre a minha solicitação. É uma completa falta de respeito com o cliente."
    ]
}
df = pd.DataFrame(data)
df

,Texto_BC
0,Fui até a agencia para falar com o gerente sob...
1,Estou há semanas aguardando um contato da agen...
2,Total falta de feedback da minha agencia. Prec...
3,"O aplicativo do banco é ótimo, mas o atendimen..."
4,"Recebi um email sobre um novo produto, mas qua..."
5,"Problema resolvido rapidamente pelo telefone, ..."
6,A agencia está sempre lotada. Pedi um retorno ...
7,Não obtive nenhuma resposta da agencia sobre a...


In [58]:
# ==============================================================================
# ETAPA 2: PRÉ-PROCESSAMENTO DO TEXTO
# ==============================================================================

def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    # Remover acentos
    nfkd_form = unicodedata.normalize('NFKD', text)
    text = "".join([c for c in nfkd_form if not unicodedata.combining(c)])
    # Remover números e caracteres especiais
    text = re.sub(r'[^a-z\s]', '', text)
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Aplicando a função e criando a nova coluna
df['text_tratado'] = df['Texto_BC'].apply(preprocess_text)
df

,Texto_BC,text_tratado
0,Fui até a agencia para falar com o gerente sob...,fui ate a agencia para falar com o gerente sob...
1,Estou há semanas aguardando um contato da agen...,estou ha semanas aguardando um contato da agen...
2,Total falta de feedback da minha agencia. Prec...,total falta de feedback da minha agencia preci...
3,"O aplicativo do banco é ótimo, mas o atendimen...",o aplicativo do banco e otimo mas o atendiment...
4,"Recebi um email sobre um novo produto, mas qua...",recebi um email sobre um novo produto mas quan...
5,"Problema resolvido rapidamente pelo telefone, ...",problema resolvido rapidamente pelo telefone n...
6,A agencia está sempre lotada. Pedi um retorno ...,a agencia esta sempre lotada pedi um retorno d...
7,Não obtive nenhuma resposta da agencia sobre a...,nao obtive nenhuma resposta da agencia sobre a...


In [59]:
# ==============================================================================
# ETAPA 3: EXPANSÃO DE FRASES COM SINÔNIMOS
# ==============================================================================

# Possíveis frases que indicam falta de retorno
frases = [
    'nao tive retorno',
    'nenhum contato',
    'falta de feedback'
]
sinonimos = [
    ['contato', 'retorno', 'feedback', 'resposta'],
    ['nenhum', 'sem', 'falta de', 'nao tive'],
    ['recebi', 'obtive', 'tive'],
    ['atendimento', 'suporte', 'apoio'],
    ['ausencia', 'falta', 'sem'],
]

# Frases que indicam espera ou demora
frases_espera = [
    'ha dias',
    'ha semanas',
    'ha meses',
    'desde o dia',
    'ate agora nada',
    'estou esperando'
]
sinonimos_espera = [
    ['ha', 'faz', 'fazem'],
    ['dia', 'dias', 'semana', 'semanas', 'mês', 'meses'],
    ['desde', 'desde o dia', 'a partir de'],
    ['ate agora nada', 'ate o momento nada', 'ate agora nenhuma resposta'],
    ['estou esperando', 'aguardando', 'fico aguardando']
]

# Frases que indicam certeza sobre a falta de retorno
frases_certeza = [
    'agencia nao me retornou',
    'agencia nao me deu resposta',
    'ate agora a agencia nao me deu respostas',
    'o gerente nao me deu retorno',
    'funcionario da agencia nao respondeu',
    'o gerente nao me respondeu',
    'ate o momento o funcionario da agencia nao me deu retorno'
]
sinonimos_certeza = [
    ['gerente', 'funcionario da agencia', 'agencia'],
    ['resposta', 'retorno', 'parecer']
]

# pre processamento
frases = [preprocess_text(fr) for fr in frases]
sinonimos = [[preprocess_text(term) for term in group] for group in sinonimos]
frases_espera = [preprocess_text(fr) for fr in frases_espera]
sinonimos_espera = [[preprocess_text(term) for term in group] for group in sinonimos_espera]
frases_certeza = [preprocess_text(fr) for fr in frases_certeza]
sinonimos_certeza = [[preprocess_text(term) for term in group] for group in sinonimos_certeza]

# Monta dicionários de lookup rápido
sinonimos_map = {palavra: grupo for grupo in sinonimos for palavra in grupo}
sinonimos_espera_map = {term: group for group in sinonimos_espera for term in group}
sinonimos_certeza_map = {palavra: grupo for grupo in sinonimos_certeza for palavra in grupo}

# Expansão de frases gerais
frases_expandidas = []
for frase in frases:
    tokens = frase.split()
    listas = [sinonimos_map.get(tok, [tok]) for tok in tokens]
    for combo in itertools.product(*listas):
        frases_expandidas.append(' '.join(combo))
frases_expandidas = list(set(frases_expandidas))

# Expansão das frases de espera
expandidas_espera = []
for frase in frases_espera:
    tokens = frase.split()
    listas = [sinonimos_espera_map.get(tok, [tok]) for tok in tokens]
    for combo in itertools.product(*listas):
        expandidas_espera.append(' '.join(combo))
expandidas_espera = list(set(expandidas_espera))

# Expansão de frases de certeza
expandidas_certeza = []
for frase in frases_certeza:
    tokens = frase.split()
    listas = [sinonimos_certeza_map.get(tok, [tok]) for tok in tokens]
    for combo in itertools.product(*listas):
        expandidas_certeza.append(' '.join(combo))
expandidas_certeza = list(set(expandidas_certeza))

print("Frases expandidas (possíveis falta de retorno):")
print(frases_expandidas)
print("\nFrases expandidas de possíveis espera:")
print(expandidas_espera)
print("\nFrases expandidas de certeza:")
print(expandidas_certeza)

Frases expandidas (possíveis falta de retorno):
['ausencia de contato', 'falta de feedback', 'nao recebi contato', 'sem de feedback', 'sem contato', 'nao obtive contato', 'falta de resposta', 'nenhum resposta', 'ausencia de resposta', 'sem de retorno', 'sem de resposta', 'falta de contato', 'nenhum retorno', 'nao obtive retorno', 'ausencia de feedback', 'nao tive retorno', 'sem de contato', 'sem resposta', 'sem retorno', 'ausencia de retorno', 'sem feedback', 'nao obtive resposta', 'nao recebi retorno', 'nao obtive feedback', 'nao recebi resposta', 'nenhum contato', 'nao tive resposta', 'nao recebi feedback', 'nao tive feedback', 'nao tive contato', 'nenhum feedback', 'falta de retorno']

Frases expandidas de possíveis espera:
['faz semanas', 'a partir de o dia', 'fazem dias', 'faz semana', 'a partir de o meses', 'a partir de o dias', 'ate agora nada', 'fazem mes', 'desde o dias', 'a partir de o semana', 'ha semanas', 'desde o meses', 'ha meses', 'desde o dia o semana', 'ha mes', 'faze

In [60]:
# ==============================================================================
# ETAPA 4: FUNÇÃO DE CÁLCULO DE SCORE
# ==============================================================================

def calculate_score(text, certeza_phrases, espera_phrases, normal_phrases):
    """
    Calcula o score de probabilidade com a regra de "certeza" e o bônus de "espera".
    """
    tokens = text.split()

    # 1. Verifica a presença de "agencia". É o gatilho principal.
    if 'agencia' not in tokens:
        return 0.0
    
    # 2. Verifica se alguma "frase de certeza" está presente.
    for phrase in certeza_phrases:
        if phrase in text:
            return 1.0

    # 3. Inicia o cálculo do score base.
    score = 0.25
    target_indices = [i for i, token in enumerate(tokens) if token == 'agencia']

    # Verifica a presença de "gerente"
    if 'gerente' in tokens:
        score += 0.25
        target_indices.extend([i for i, token in enumerate(tokens) if token == 'gerente'])

    # 4. (NOVO) Verifica a presença de frases de "espera" e adiciona um bônus.
    for phrase in espera_phrases:
        if phrase in text:
            score += 0.15
            break # Adiciona o bônus uma vez e para de procurar.

    # 5. Verifica a presença de frases normais e calcula score de proximidade.
    max_phrase_score = 0.0
    for phrase in normal_phrases:
        phrase_tokens = phrase.split()
        len_phrase = len(phrase_tokens)
        
        for i in range(len(tokens) - len_phrase + 1):
            if tokens[i:i+len_phrase] == phrase_tokens:
                phrase_start_index = i
                min_dist = float('inf')
                for target_idx in target_indices:
                    dist = abs(phrase_start_index - target_idx) -1
                    if phrase_start_index + len_phrase <= target_idx:
                        dist = target_idx - (phrase_start_index + len_phrase)
                    elif phrase_start_index > target_idx:
                        dist = phrase_start_index - (target_idx + 1)
                    if dist < min_dist:
                        min_dist = dist
                
                current_phrase_score = 0.0
                if min_dist <= 2:
                    current_phrase_score = 0.5
                else:
                    decay_factor = 0.05 
                    current_phrase_score = max(0, 0.5 - (min_dist - 2) * decay_factor)
                
                if current_phrase_score > max_phrase_score:
                    max_phrase_score = current_phrase_score

    final_score = score + max_phrase_score
    return min(final_score, 1.0)

texto_exemplo = df['text_tratado'][0]
score_exemplo = calculate_score(texto_exemplo, expandidas_certeza, expandidas_espera, frases_expandidas)
print(f"Texto: '{texto_exemplo}'")
print(f"Score Calculado: {score_exemplo:.2f}")

Texto: 'fui ate a agencia para falar com o gerente sobre o meu problema mas nao tive retorno algum sobre o caso'
Score Calculado: 0.85


In [61]:
# ==============================================================================
# ETAPA 5: APLICAÇÃO DO PIPELINE E ANÁLISE DOS RESULTADOS
# ==============================================================================

df['probabilidade_score'] = df['text_tratado'].apply(
    lambda text: calculate_score(
        text, 
        expandidas_certeza, 
        expandidas_espera, 
        frases_expandidas
    )
)

# Ordenando por score para melhor visualização
df_resultado = df.sort_values(by='probabilidade_score', ascending=False)

df_resultado

,Texto_BC,text_tratado,probabilidade_score
0,Fui até a agencia para falar com o gerente sob...,fui ate a agencia para falar com o gerente sob...,0.85
2,Total falta de feedback da minha agencia. Prec...,total falta de feedback da minha agencia preci...,0.75
1,Estou há semanas aguardando um contato da agen...,estou ha semanas aguardando um contato da agen...,0.65
6,A agencia está sempre lotada. Pedi um retorno ...,a agencia esta sempre lotada pedi um retorno d...,0.65
3,"O aplicativo do banco é ótimo, mas o atendimen...",o aplicativo do banco e otimo mas o atendiment...,0.50
4,"Recebi um email sobre um novo produto, mas qua...",recebi um email sobre um novo produto mas quan...,0.25
7,Não obtive nenhuma resposta da agencia sobre a...,nao obtive nenhuma resposta da agencia sobre a...,0.25
5,"Problema resolvido rapidamente pelo telefone, ...",problema resolvido rapidamente pelo telefone n...,0.00
